In [1]:
# import required modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc

In [4]:
# import metadata matrix
mm = pd.read_csv('../data_tables/metadataMatrix.csv')
mm

,journal,pmc_ID,accession,converted_accession,repository,pmc_date,repository_date,species,hardware,library_strategy,sraAvg_length,sraBases,sraAccess,total_use_count
0,Alzheimers_Res_Ther,PMC3707052,GSM1,GSE506,GEO,2013,"Sep 28, 2000",Homo sapiens,SAGE NlaIII,Expression_Array,NaN,NaN,NaN,64
1,Alzheimers_Res_Ther,PMC3706879,GSE45534,GSE45534,GEO,2013,"May 25, 2004",Mus musculus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1
2,Alzheimers_Res_Ther,PMC4255636,GSE5281,GSE5281,GEO,2014,"Nov 07, 2003",Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,43
3,Alzheimers_Res_Ther,PMC4731966,GSE67036,GSE67036,GEO,2016,"May 06, 2013",Rattus norvegicus,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1
4,Alzheimers_Res_Ther,PMC4731966,GSE1297,GSE1297,GEO,2016,"Mar 11, 2002",Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73928,Dis_Markers,PMC6589307,GSE14795,GSE14795,GEO,2019,"Mar 11, 2002",Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,1
73929,Dis_Markers,PMC6589301,GSE62254,GSE62254,GEO,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
73930,Dis_Markers,PMC3834650,GDS1059,GSE2191,GEO,2013,"Mar 16, 2009",Homo sapiens,in situ oligonucleotide,Expression_Array,NaN,NaN,NaN,5
73931,Case_Rep_Hematol,PMC6462343,PRJNA437812,SRP135146,SRA,2019,2018-03-13 01:47:12,human gut metagenome,Illumina MiSeq,AMPLICON,602.0,17743348.0,public,1


In [5]:
# define function defining each paper as a GEO or SRA paper...
# ...based on which is mentioned more, or 'Both' if score is equal
def getRepo(paper_IDs):
    try:
        r = statistics.mode(paper_IDs)
    except:
        r = 'Both'
    return r

In [7]:
# Look at each PAPER
# Across GEO and SRA, what percentage of papers reuse data? (stacked bar chart)

# Take a slice of allFactors with the data we want
byPaper = mm[['pmc_ID', 'repository', 'total_use_count']]
byPaper = byPaper.dropna(subset = ['repository'])
byPaper = byPaper.groupby(['pmc_ID'], as_index = False).agg({
    'repository': lambda x: getRepo(x), 'total_use_count': 'mean'})
byPaper.columns = ['pmc_ID', 'repository', 'avg_use']

# divide into GEO, SRA, Both
byPaperGEO = byPaper.loc[byPaper['repository'] == 'GEO']
byPaperSRA = byPaper.loc[byPaper['repository'] == 'SRA']
byPaperBOTH = byPaper.loc[byPaper['repository'] == 'Both']

byPaper

,pmc_ID,repository,avg_use
0,PMC1079818,Both,1.000000
1,PMC1082884,Both,2.000000
2,PMC1088941,Both,2.000000
3,PMC1088946,Both,2.000000
4,PMC1090574,Both,1.400000
...,...,...,...
26065,PMC6639121,Both,2.000000
26066,PMC6639415,Both,1.666667
26067,PMC6640630,Both,1.000000
26068,PMC6646343,Both,1.000000


In [ ]:
tc = Counter(byPaper['avg_use'])
totalCounts = pd.DataFrame.from_dict(tc, orient = 'index').reset_index()

gc = Counter(byPaperGEO['avg_use'])
geoCounts = pd.DataFrame.from_dict(gc, orient = 'index').reset_index()

sc = Counter(byPaperSRA['avg_use'])
sraCounts = pd.DataFrame.from_dict(sc, orient = 'index').reset_index()

bc = Counter(byPaperBOTH['avg_use'])
bothCounts = pd.DataFrame.from_dict(bc, orient = 'index').reset_index()

for df in [totalCounts, geoCounts, sraCounts, bothCounts]:
    df.columns = ['avg_use', 'num_papers']
    df = df.sort_values(by = 'times_used').reset_index().drop(labels = ['index'], axis = 1)
    # no_reuse is number of papers only mentioning datasets used once
    no_reuse = df['num_papers'][0]
    # tot_papers includes all papers, including those that mentioned at least one reused dataset
    tot_papers = df['num_papers'].sum()
    pct_reuse_papers = 1 - no_reuse/tot_papers
    print(pct_reuse_papers)